## Setup Repo

In [ ]:
!git clone https://github.com/Federated-Learning-S3U/BiseNet-FL.git

In [ ]:
!mkdir -p res

In [ ]:
%cd /kaggle/working/BiseNet-FL

## Managing Dependencies 

In [ ]:
# Uninstall scikit-learn
!pip uninstall scikit-learn -y

# Reinstall compatible version for your current numpy
!pip install 'scikit-learn>=1.6.0' --no-deps --force-reinstall

In [ ]:
# !pip3 install -e . --force-reinstall
!pip install -e .

## Setup Configs
- Mainly you will only need to copy paste the pytoml file in fl-cityscapes-bisenetv2/fl_cityscapes_bisenetv2/configs.
- On resume, make sure to add the previous model as an input, and map the path for it accordingly.
- In addition, make sure to modify the required parameters tagged with IMPORTANT.

In [ ]:
file_path = "/kaggle/working/BiseNet-FL/fl-cityscapes-bisenetv2/pyproject.toml"

toml_content = """
# FLWR TOML CONFIG FILE FOR CITYSCAPES WITH CITY PARTITIONS

# Dataset Setup: 18 clients,
# average of 165 image per client,
# partition is considered to be iid.

[build-system]
requires = ["hatchling"]
build-backend = "hatchling.build"

[project]
name = "fl-cityscapes-bisenetv2"
version = "1.0.0"
description = ""
license = "Apache-2.0"

[tool.hatch.build.targets.wheel]
packages = ["."]

[tool.flwr.app]
publisher = "moustafa"

# Point to your ServerApp and ClientApp objects
[tool.flwr.app.components]
serverapp = "fl_cityscapes_bisenetv2.server_app:app"
clientapp = "fl_cityscapes_bisenetv2.client_app:app"

# Custom config values accessible via `context.run_config`
[tool.flwr.app.config]

# Server-side FL configs
server-device = "cuda"
strategy-name = "FedAvg" # Options: FedAvg, FedAvgM, FedProx
num-server-rounds = 60
fraction-train = 0.5
fraction-evaluate = 0.0

# Server-side Resume Training Configs
# IMPORTANT UPDATE ON RESUME
resume = true
pretrained-path = "/kaggle/input/city-partitions/pytorch/fedavg-60/2/latest_model.pt" # Might need to be added as an input model
rounds-trained = 60

# Strategy-specific Configs
# FedAvgM Configs
server-momentum = 0.95
server-learning-rate = 1.0

# FedProx Configs
proximity-mu = 0.005

# Server-side Evaluation Configs
eval-batch-size = 8
eval-interval = 1
server-data-partition = "./datasets/cityscapes/val.txt"

# Server-side Saving Configs
save-latest = "/kaggle/working/res/latest_model.pt"
save-best = "/kaggle/working/res/best_model.pt"
server-results-file = "/kaggle/working/res/server_results.txt"

best-miou = 0.38107714519821506 # IMPORTANT UPDATE ON RESUME
best-metric = "/kaggle/working/res/best_metric.json"
latest-metric = "/kaggle/working/res/latest_metric.json"

# Client-side FL Configs
client-device = "cuda"

# Client Dataset Paths
im-root = "/kaggle/input/cityscapes-fine-dataset"
client-data-partition = "./datasets/cityscapes/city_partitions.json"

# Model Training Hyper-Params
local-epochs = 3 # Note, on increasing local epochs may require reducing the LR
batch-size = 8 # Fix batch for memory constraints

lr = 0.02657205000000001 # IMPORTANT UPDATE ON RESUME
lr-decay-factor = 0.9
lr-decay-rounds = 10
lr-schedule-file = "/kaggle/working/res/lr_schedule.json"

# Model Configs
num-classes = 19
lb-ignore = 255
num-aux-heads = 4

# Optimizer Configs
weight-decay = 5e-4

# Data Augmentation Configs
scales = "[0.25, 2.0]"
cropsize = "[512, 1024]"

# Default federation to use when running the app
[tool.flwr.federations]
default = "local-simulation-single-gpu"

# Local simulation federation with 18 virtual SuperNodes
[tool.flwr.federations.local-simulation-single-gpu]
options.backend.init_args.num_cpus = 2 # Number of CPUs for the simulation controller 
options.backend.init_args.num_gpus = 1 # Number of GPUs for the simulation controller
options.num-supernodes = 18 # Number of SuperNodes (i.e., Clients) in the Data Setup
options.backend.client-resources.num-cpus = 2 # Number of CPUs per Client
options.backend.client-resources.num-gpus = 1.0 # Fractional GPU allocation per Client
"""

with open(file_path, "w") as f:
    f.write(toml_content.strip())

print(f"File successfully written to: {file_path}")

## Run Simulation

In [ ]:
# Ensure To be inside BiseNet-FL
!flwr run fl-cityscapes-bisenetv2

## Save Zipped Results For Download
- After downloading the results from notebook output, you will need to download the zipped file, then upload it to the drive.

In [ ]:
!zip -r /kaggle/working/run_results.zip /kaggle/working/res